# Finding Lane Lines

Features: Colour, Shape, Orientation, Position in the image.
1. Colour
    - e.g. RGB: [255,255,255] (Three colour channels, 0 is darkest and 255 is brightest.)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# Variables
image_name='test.jpg'

# Read in the image and print out some stats
image = mpimg.imread(image_name)
print('This image is: ', type(image), ' with dimensions: ', image.shape)

# Grab the x and y size and amke a copy of the image
# Think of it as a matrix so rows are y and cols are x
ysize = image.shape[0]
xsize = image.shape[1]
color_select = np.copy(image)

# Define our colour selection criteria.
# I.e. the minimum values for RGB that we will allow in our selection.
red_threshold = 240
green_threshold = 240
blue_threshold = 240
rgb_threshold = [red_threshold, green_threshold, blue_threshold]

# Define criteria to select any pixels below the threshold
# Using a 'bitwise OR'
# Red component of pixel is less than red threshold or G...or B...
thresholds = (image[:,:,0] < rgb_threshold[0]) \
            | (image[:,:,1] < rgb_threshold[1]) \
            | (image[:,:,2 < rgb_threshold[2]])

# Set any pixels below the threshold to zero (black) in copy of 
# image `color_select'
color_select[thresholds] = [0,0,0]

# Resulting image: Pixels above the threshold (brighter) have been retained,
# pixels below the threshold blacked out.
plt.imshow(color_select)

'In this case, I'll assume that the front facing camera that took the image is mounted in a fixed position on the car, such that the lane lines will always appear in the same general region of the image. Next, I'll take advantage of this by adding a criterion to only consider pixels for color selection in the region where we expect to find the lane lines.'

In [ ]:
# Coding up a region of interest mask

# Define a triangular region of interest
# [y_from_top_down, x_from_left_to_right]
# Dimensions of this image: 540 (rows), 960 (cols), 3
left_bottom = [120, 540]
right_bottom = [800, 540]
apex = [480, 300]

# Fit lines (y=Ax+B) to identify the 3-sided region of interest.
# np.polyfit() returns the coefficients [A, B] of the fit.
# Left line, i.e. line between `left_bottom` and `apex`
# np.polyfit(x, y, deg_of_fitting_polynomial)
fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

# Find the region inside the lines
# np.meshgrid returns coordinate matrices from coordinate vectors.
# returns two arrays of arrays
XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                    (YY > (XX*fit_right[0] + fit_right[1])) & \
                    (YY < (XX*fit_bottom[0] + fit_bottom[1]))

# Find where image is both coloured right and in the region
region_select[~region_thresholds] = [255,0,0]

# Display the image
plt.imshow(region_select)

In [ ]:
line_image = np.copy(image)

# Find where the image is both coloured right and in the region
# ~ colour_thresholds: selects pixels BRIGHTER than the colour thershold
# region_thresholds: selects pixel IN the region
# Colour these pixels red.
line_image[~color_thresholds & region_thresholds] = [255,0,0]

plt.imshow(line_image)

In [6]:
XX, YY = np.meshgrid(np.arange(0,100), np.arange(0,100))

In [9]:
np.meshgrid(np.arange(0,100), np.arange(0,100))

[array([[ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        ..., 
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99]]), array([[ 0,  0,  0, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 2,  2,  2, ...,  2,  2,  2],
        ..., 
        [97, 97, 97, ..., 97, 97, 97],
        [98, 98, 98, ..., 98, 98, 98],
        [99, 99, 99, ..., 99, 99, 99]])]

In [7]:
XX

array([[ 0,  1,  2, ..., 97, 98, 99],
       [ 0,  1,  2, ..., 97, 98, 99],
       [ 0,  1,  2, ..., 97, 98, 99],
       ..., 
       [ 0,  1,  2, ..., 97, 98, 99],
       [ 0,  1,  2, ..., 97, 98, 99],
       [ 0,  1,  2, ..., 97, 98, 99]])

In [8]:
YY

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 2,  2,  2, ...,  2,  2,  2],
       ..., 
       [97, 97, 97, ..., 97, 97, 97],
       [98, 98, 98, ..., 98, 98, 98],
       [99, 99, 99, ..., 99, 99, 99]])

Computer vision: using algorithms to let the computer see the world like we do. Depth, colour, shapes, meaning.

### Canny Edge Detection
Identifying the boundaries of an object in an image
- Convert to grayscale
- Compute gradient (brighter pixel -> stronger gradient)
- Find edges by identifying pixels wwith strongest gradient.

#### Canny algorithm
Computing gradient gives us thick edges. -> Canny algorithm thins edges to individual pixels that have strongest gradients.

```
edges = cv2.Canny(gray, low_threshold, high_threshold)
```

Summary
- Apply Canny to image `gray` and return binary (B&W) image `edges`.
- Edges will be white and the rest of the image will be black.

Steps:
1. Alg first detects strong gradient pixels above the `high_threshold` and rejects pixels below the `low_threshold` value. 
2. Next, pixels with values between the `low_threshold` and `high_threshold` will be included as long as they are connected to strong edges.

#### Reasonable range for parameters
- E.g. In an 8-bit image, each pixel has 256 possible values (0-255). So the derivatives will be on the scale of tens to hundreds. 
    - Range for threshold parameters: tens to hundreds
- Ratio `low_threshold:high_threshold`: 1:2 or 1:3 (John Canny)

#### Gaussian smoothing to suppress noise and spurious gradients
- Larger `kernel_size` -> smoothing over a larger area.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2


# Read in image
image = mpimg.imread('exit_ramp.jpg')
# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Show image
plt.imshow(gray, cmap='gray')


# Define a kernel size for Gaussian smoothing / blurring
# Kernel size must be an odd number.
kernel_size = 3
# Run Gaussian smoothing
blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size), 0)


# Define parameters for Canny edge detector
low_threshold = 50
high_threshold = 150
# Run Canny edge detector on the image
edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

plt.imshow(edges, cmap='Greys_r')

### Hough transform
- Motivation: Previously have image of points. Need to find lines.
- Hough Transform: Conversion from image space to Hough space (space in parameter m, b of line). 
    - Each line in image space is then a point in Hough space.
    - Each point in image space is a line in Hough space. -.-
    - Two points in image space correspond to two intersecting lines in Hough Space.
    - The intersection point of two lines in Hough space corresponds to a line in image space that passes through 
- Look for intersecting lines in Hough space to find lines in image space.
- Do this by dividing Hough Space into grid and define intersecting lines as all lines passing through a grid cell.
- But cannot represent vertical lines (infinite slope) in m,b representation. So represent in **polar coordinates** (theta, rho) instead.
    - Points in image space become sine curves in Hough Space. Okaay?

Use OpenCV function **HoughesLineP** to specify parameters to say what kind of lines we want to detect (long, short, bendy, dashed lines).